In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
reviews = pd.read_csv('encoded_reviews_file.csv')
reviews

,lemmatized,Sentiment
0,review give 24 hrs l find amazing 10 day usage...,1
1,like tell win t give technical review like ytu...,2
2,receive unit 4 hr ago think 1 feel sturdy prem...,2
3,get oneplus nord 2 5 g phone deliver yesterday...,1
4,7 28 receive yesterday impression look feel gr...,0
...,...,...
5597,camera bad,0
5598,camera stop work month fast charging isn t wor...,0
5599,set clone app whatsapp doneclone whatsapp pr k...,0
5600,phone lag 22 storage,0


# User Defined Functions